In [ ]:
# Bước 1: Đọc dữ liệu từ các file đã chia
import pandas as pd

# Đọc các file train/val/test
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

print(train_df.head())

In [ ]:
# Bước 2: Chuyển dữ liệu về dạng phù hợp với từng mô hình
train_df['Date'] = pd.to_datetime(train_df['Date'])  # cột Date
train_df.set_index('Date', inplace=True)
